Import packages.

In [1]:
import scipy
import numpy as np
import matplotlib.pyplot as plt
from typing import Callable

## The Backward Euler method

We analyze the semi-batch reactor. The reactor model contains a non-autonomous term (right hand-side contains the independent variable $t$ explicitly).

$\frac{dC_A}{dt}=\frac{\dot{V}}{V_0+\dot{V}t}\left(C_A^{in}-C_A\right)-kC_A$

$\frac{dC_B}{dt}=kC_A-\frac{\dot{V}}{V_0+\dot{V}t}C_B$

For this example we set:

$ C_A^{in} = 0.5 \ mol/L$

$ V_0 = 10 \  L$

$ \dot{V} = 0.1 \ L/ min $

$ k = 0.2 \ min^{-1} $

Initial conditions:

$C_A(t=0\,s)=1\, mol/L,\; C_B(t=0\,s)=0\, mol/L$

Define parameters.

In [2]:
# time domain
t_start = 0.0
t_end = 30.0
i = 500
t = np.linspace(t_start, t_end, i)

# initial conditions
c0 = np.array([1,0])

Define chemical reaction system.

In [3]:
def dC(t: float, c: np.ndarray) -> np.ndarray:
    """ODE system for semi-batch reactor.

    Parameters
    ----------
    t : float
        Time variable
    c : np.ndarray
        Concentration of individual components

    Returns
    -------
    np.ndarray
        Concentration gradient in reactor
    """
    cA, cB = c

    # define constant parameters
    k = 0.2       
    Vdot = 0.1    
    V_0 = 10
    cA_in = 0.5

    # define the system of ODEs
    dcA = Vdot/(V_0+Vdot*t)*(cA_in-cA)-k*cA
    dcB = k*cA-Vdot/(V_0+Vdot*t)*cB
    
    return np.array([dcA, dcB])

Define backward Euler method.

We use a 2D array to store the concentrations: $c = \begin{pmatrix} c_{A,0} & c_{A,1} & \dots & c_{A,N-1} \\ c_{B,0} & c_{B,1} & \dots & c_{B,N-1}  \end{pmatrix}$

In [4]:
def backward_euler(func: Callable, c0: np.ndarray, t: np.ndarray) -> np.ndarray:
    """Generic backward Euler method for initial value problem. Use scipy's 
    fsolve to solve root finding problem.

    Parameters
    ----------
    func : Callable
        Function that defines the ODE (y' = func(t, y)).
    c0 : np.ndarray
        Initial condition.
    t : np.ndarray
        Time domain.

    Returns
    -------
    np.ndarray
        Array of solution values at the time points.
    """
    # initialize arrays for time and solution values
    c = np.zeros([len(c0),len(t)])
    dt = t[1]-t[0]

    # initial condition
    c[:,0] = c0
    
    # iterate over each time step
    for i in range(len(t)-1):
        
        # initial guess for y_{i+1}
        c_guess = c[:,i]

        # define backward Euler function
        euler = lambda c_next: c[:,i] + dt * func(t[i+1], c_next) - c_next

        # update solution
        c[:,i+1] = scipy.optimize.fsolve(euler, c_guess)
    
    return c

Execute backward Euler method.

In [5]:
c = backward_euler(dC, c0, t)

Plot the results.

In [ ]:
fig = plt.figure()
plt.plot(t, c[0,:], label = "c$_A$")
plt.plot(t, c[1,:], label = "c$_B$")
plt.xlabel("time [s]")
plt.ylabel("concentration [mol/L]")
plt.title("The Backward Euler method")
plt.legend()
plt.grid()
plt.show()